In [ ]:
se carga el dataset ya balanceado y el vectorizador TF-IDF que se generó en el notebook anterior

In [2]:
import pandas as pd
import joblib

df = pd.read_csv("/content/buzon_balanceado.csv")
vectorizador = joblib.load("/content/tfidf_vectorizer.pkl")

X = vectorizador.transform(df["comentario"])
y = df["etiqueta"]




aquí empieza la parte del entretamiento donde se dividen los datos en un conjunto de entrenamiento 80% y uno de prueba 20%

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


se entrena el modelo de regresión logística utilizando el parámetro class_weight="balanced" para dar mayor importancia a las clases menos frecuentes

In [4]:
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(
    class_weight="balanced",
    max_iter=1000
)

modelo.fit(X_train, y_train)



LogisticRegression(class_weight='balanced', max_iter=1000)

se genera la matriz de ocnfusión y el reporte de clasificación para analizar las métricas comúnes.

Para la clase *felicitación*, el modelo clasificó correctamente 14 comentarios. Sin embargo, 3 comentarios
que realmente eran felicitaciones fueron clasificados como quejas. No se observaron confusiones con la clase sugerencia,
lo que indica que el modelo distingue bien el lenguaje claramente positivo.

En la clase *queja*, 7 comentarios fueron clasificados correctamente. No obstante, 1 comentario fue confundido
con una felicitación y 2 con sugerencias. Esto sugiere que algunas quejas presentan un tono moderado que puede
parecer una sugerencia o incluso un comentario neutral.

En la clase *sugerencia*, el modelo clasificó correctamente 7 comentarios. Sin embargo, 6 sugerencias fueron
confundidas con quejas y 2 con felicitaciones. Esta es la clase con mayor nivel de confusión, lo cual es esperable
debido a que las sugerencias pueden contener críticas implícitas similares a las quejas.

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = modelo.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))



[[14  3  0]
 [ 1  7  2]
 [ 2  6  7]]
              precision    recall  f1-score   support

felicitación       0.82      0.82      0.82        17
       queja       0.44      0.70      0.54        10
  sugerencia       0.78      0.47      0.58        15

    accuracy                           0.67        42
   macro avg       0.68      0.66      0.65        42
weighted avg       0.72      0.67      0.67        42



In [6]:
joblib.dump(modelo, "/content/clasificador.pkl")



['/content/clasificador.pkl']

se diseña una función de predicción que recibe un texto cualquiera, lo procesa y devuelve la etiqueta predicha por el modelo

In [7]:
def clasificar_comentario(texto):
    texto = texto.lower().strip()
    X = vectorizador.transform([texto])
    return modelo.predict(X)[0]


In [8]:
print(clasificar_comentario("Muchas gracias por la excelente atención"))
print(clasificar_comentario("El trámite fue muy lento y no resolvieron nada"))
print(clasificar_comentario("Sería bueno ampliar el horario de atención"))


felicitación
queja
sugerencia


se implementa una función para visualizar qué tan seguro está el modelo de su predicción, mostrando el porcentaje de pertenencia a cada clase.

El modelo no solo asigna una clase final, sino que también devuelve probabilidades asociadas a cada clase.
Por ejemplo, para un comentario dado se obtuvieron las siguientes probabilidades:

Felicitación: 0.16  
Queja: 0.57  
Sugerencia: 0.26  

Esto indica que el comentario tiene mayor afinidad con la clase queja, aunque también presenta características
asociadas a sugerencia. Esta ambigüedad es consistente con la confusión observada en la matriz de confusión.

El uso de probabilidades permite identificar comentarios ambiguos y puede ser aprovechado para establecer
umbrales de decisión o para alimentar modelos posteriores


In [9]:
def clasificar_con_prob(texto):
    X = vectorizador.transform([texto])
    clases = modelo.classes_
    probs = modelo.predict_proba(X)[0]
    return dict(zip(clases, probs))

print(clasificar_con_prob("El sistema no funciona y nadie responde"))


{'felicitación': np.float64(0.16470812637872181), 'queja': np.float64(0.5728564279780808), 'sugerencia': np.float64(0.2624354456431973)}


El modelo logra cumplir adecuadamente su objetivo principal: clasificar automáticamente los comentarios
en felicitación, queja o sugerencia al momento de ser recibidos.

La clase felicitación es la mejor aprendida, mientras que la mayor dificultad se presenta al distinguir
entre sugerencias y quejas, debido a similitudes semánticas en el lenguaje utilizado.

A pesar de estas limitaciones, el modelo proporciona una base sólida para la segunda etapa del sistema,
en la cual los comentarios pueden ser canalizados al departamento correspondiente, utilizando tanto
la clase predicha como las probabilidades asociadas.